In [61]:
from speak_note.tools.llms import llm_list
from speak_note.tools.context import retriever_configs, context_configs, Context
from speak_note.work_flows import basic_RAG
from speak_note.tools.myPDFparser import upstageParser2Document
import speak_note.prompts.prompt as  prompt

import asyncio

# 서버 첫 INIT
### 해야할일
1. 문서pdf -> 문서json
2. 메타데이터에 {문서요약 : 문서json경로} append하기.

## meta_data.json 생성

### ㅋㅋ

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt_to_summarize = """
    당신은 행정문서를 한줄 요약해주는 어시스턴트입니다.
    문서의 정보를 반영한 최대한 짧고 간결한 문서 요약내용을 제시하세요.
    이모티콘, 기호 사용금지.

    #User Input (요약 요청 내용):
    {question}

    #요약 내용:
    """

basic_prompt_template = PromptTemplate.from_template(prompt_to_summarize)
llm = llm_list["solar-pro2"]

chain = (
    basic_prompt_template
    | llm
    | StrOutputParser()
)

In [63]:
import os
from pathlib import Path
import asyncio
import json

# 경로 준비
data_root_str = "/Users/yujin/Desktop/코딩shit/python_projects/대한민국해커톤/민원마루ver1/datas"
rawDocs_name  = "rawDocs"
parsed_docs_name = "parsedDos"

data_root = Path(data_root_str)
raw_docs_root = data_root.joinpath(rawDocs_name)
parsed_docs_root = data_root.joinpath(parsed_docs_name)

json_paths = [f for f in parsed_docs_root.rglob("*.json") if f.is_file()]
docs_num = len(json_paths)

In [64]:
from speak_note.tools.json_tool import load_json

docs = [(json_paths[i], load_json(json_path)) for i, json_path in enumerate(json_paths)]
print(docs[0][0])


/Users/yujin/Desktop/코딩shit/python_projects/대한민국해커톤/민원마루ver1/datas/parsedDos/sample_AI_Brief.json


In [ ]:
def get_str_from_doc(doc_contents):
    page_content = ""
    for idx in range(len(doc_contents)):
        contents = doc_contents[idx].to_json()["kwargs"]["page_content"]
        page_content = page_content + "\n" + contents
    return page_content

# doc_path = docs[0][0]
# doc_content = get_str_from_doc(docs[0][1])
# print(doc_path)

/Users/yujin/Desktop/코딩shit/python_projects/대한민국해커톤/민원마루ver1/datas/parsedDos/sample_AI_Brief.json


In [66]:
doc_contents = [get_str_from_doc(doc) for doc_path, doc in docs]

In [67]:
task = [chain.ainvoke(doc_content) for doc_content in doc_contents]
results = await asyncio.gather(*task)

In [68]:
summarize_path_tuple = [(results[i], json_paths[i]) for i in range(docs_num)]
summarize_path_tuple[0]

("1. **정책/법제**  \n미국, '안전하고 신뢰할 수 있는 AI 개발 및 사용' 행정명령 발표(2023.10.30). AI 안전·보안 기준 강화, 개인정보 보호, 형평성·시민권 향상, 소비자 보호, 노동자 지원, 혁신 촉진, 국제협력 등을 주요 내용으로 함. G7, '히로시마 AI 프로세스' 통해 첨단 AI 시스템 위험 관리 국제 행동강령 합의(2023.10.30). 영국 주최 AI 안전성 정상회의에서 28개국, '블레츨리 선언' 채택(2023.11.01-02). 첨단 AI 안전 테스트 협력 및 투명성 강화 합의.  \n\n2. **기업/산업**  \n첨단 AI 개발 기업은 AI 시스템 안전성 평가·공유 의무 부과(미국). G7 행동강령에 따라 AI 수명주기 전반 위험 평가·완화, 보안 통제, 콘텐츠 출처 확인 등 이행 권고.  \n\n3. **기술/연구**  \n미국, 국가AI연구자원(NAIRR) 구축으로 AI 연구 인프라 지원 확대. 영국 AI 안전 연구소 주도 첨단 AI 안전 테스트 계획 발표. '과학의 현황' 보고서 작성으로 첨단 AI 위험 평가 및 연구 우선순위 제시.  \n\n4. **인력/교육**  \n미국, AI 분야 전문가 유치를 위한 비자 절차 간소화. G7 행동강령에서 산업·정부·학계 협력 및 정보공유 강조. 영국은 공공 부문 역량 구축 협력 계획 수립.",
 PosixPath('/Users/yujin/Desktop/코딩shit/python_projects/대한민국해커톤/민원마루ver1/datas/parsedDos/sample_AI_Brief.json'))

In [ ]:
server_id = "abc"

''' 
{
    DocNum:3,
    serverID:as2f,
    Docs : {
        {"문서1 요약 내용" : minwon_maru/datas/rawDocs/문서1.json }, # 문서1의 위치
        {"문서2 요약 내용" : minwon_maru/datas/rawDocs/문서2.json },
        {"문서3 요약 내용" : minwon_maru/datas/rawDocs/문서3.json },
    }


'''

In [82]:
import asyncio
import json
from pathlib import Path
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

from speak_note.tools.json_tool import load_json

prompt_to_summarize = """   
    당신은 행정문서를 한줄 요약해주는 어시스턴트입니다.
    문서의 정보를 반영한 최대한 짧고 간결한 문서 요약내용을 제시하세요.
    이모티콘, 기호 사용금지.

    #User Input (요약 요청 내용):
    {question}

    #요약 내용:
    """

def get_str_from_doc(doc_contents):
    """여러 Document 객체 리스트에서 page_content만 합쳐서 반환"""
    page_content = ""
    for idx in range(len(doc_contents)):
        contents = doc_contents[idx].to_json()["kwargs"]["page_content"]
        page_content = page_content + "\n" + contents
    return page_content


async def generate_metadata(
    server_id: str,
    data_root_str: str = "/Users/yujin/Desktop/코딩shit/python_projects/대한민국해커톤/민원마루ver1/datas",
    parsed_docs_name: str = "parsedDos",
    llm = llm_list["solar-pro2"],
    prompt_to_summarize: str = prompt_to_summarize,
) -> dict:
    """
    주어진 파라미터에 따라 문서를 읽고 요약 후 metadata 생성.

    Args:
        server_id: 서버 식별자
        data_root_str: 데이터 루트 경로
        parsed_docs_name: JSON이 저장된 폴더 이름
        llm: 사용할 LLM (예: llm_list["solar-pro2"])
        prompt_to_summarize: 요약용 프롬프트 문자열

    Returns:
        metadata 딕셔너리
    """

    # 경로 세팅
    data_root = Path(data_root_str)
    metadata_path = data_root.joinpath("metadata.json")
    parsed_docs_root = data_root.joinpath(parsed_docs_name)

    # JSON 파일 불러오기
    json_paths = [f for f in parsed_docs_root.rglob("*.json") if f.is_file()]
    docs = [(json_path, load_json(json_path)) for json_path in json_paths]

    # Document → 문자열 변환
    doc_contents = [get_str_from_doc(doc) for _, doc in docs]

    # Chain 준비
    basic_prompt_template = PromptTemplate.from_template(prompt_to_summarize)
    chain = basic_prompt_template | llm | StrOutputParser()

    # 비동기 요약 태스크 실행
    tasks = [chain.ainvoke(doc_content) for doc_content in doc_contents]
    results = await asyncio.gather(*tasks)

    # metadata 생성
    docs_list = []
    for summary, path in zip(results, json_paths):
        docs_list.append({summary: str(path)})

    metadata = {
        "DocNum": len(results),
        "serverID": server_id,
        "Docs": docs_list,
    }

    with metadata_path.open("w", encoding="utf-8") as f:
        json.dump(metadata, f, ensure_ascii=False, indent=2)

    return metadata


In [83]:
metadata = await generate_metadata(server_id)
metadata

{'DocNum': 5,
 'serverID': 'abc',
 'Docs': [{"1. **정책/법제**: 미국, AI 안전·보안 기준, 개인정보 보호, 형평성, 소비자 보호, 노동자 지원, 국제협력 등을 포함한 행정명령 발표. G7, 첨단 AI 시스템 위험 관리 국제 행동강령 채택. 28개국, AI 안전 보장을 위한 '블레츨리 선언' 합의.  \n2. **기업/산업**: AI 개발 기업, 안전 테스트 결과 및 주요 정보 공유 의무화. G7 행동강령 자발적 채택 권고. 첨단 AI 기업, 투명성 강화 및 안전 평가 책임 부여.  \n3. **기술/연구**: AI 수명주기 위험 평가·완화, 콘텐츠 인증 메커니즘 개발, 국가AI연구자원(NAIRR)을 통한 연구 인프라 지원.  \n4. **인력/교육**: AI 분야 외국인 전문가 비자 절차 간소화. 공공부문 역량 구축 및 과학 연구개발 협력.  \n\n(요약: 주요국이 AI 안전·윤리 기준 수립 및 국제협력 강화, 기업 책임 강조, 기술 연구·인력 지원 방안 마련)": '/Users/yujin/Desktop/코딩shit/python_projects/대한민국해커톤/민원마루ver1/datas/parsedDos/sample_AI_Brief.json'},
  {'부산광역시 금정구 도서관 운영 규정은 회원 자격, 가입 절차, 대출 및 반납, 회원 의무, 탈퇴, 재가입, 자료 분실·훼손 시 변상, 장애인 도서 택배 서비스 등을 규정하며, 2024년 6월 14일 개정 시행되었다. 주요 내용은 부산광역시 거주자 및 재직자 회원 자격 부여, 14세 미만 아동 보호자 동의 필요, 전자도서관 별도 가입, 대출 기간 15일 및 재대출 조건, 미반납 시 제재, 자료 분실·훼손 시 변상 의무 등을 포함한다.  \n\n(요약 핵심: 금정구 도서관 회원 및 자료 관리 절차, 대출 조건, 개정 사항)': '/Users/yujin/Desktop/코딩shit/python_projects/대한민국해커톤/민원마루ver1/datas/parsedDos/부산

In [ ]:
from speak_note.engine.init.make_metadata import generate_metadata

sever_id = "111"